In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Conv2D, Dropout, MaxPool2D, Flatten
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, r2_score
import numpy as np

Using TensorFlow backend.


In [51]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
submission = pd.read_csv('sample_submission.csv', index_col=0)

In [52]:
train = train.iloc[:,36:]

In [53]:
x = train.iloc[:,:-4]
y = train.iloc[:,-4:]

In [54]:
test = test.iloc[:,36:]

In [55]:
x = x.interpolate(axis=0)
test = test.interpolate(axis=0)

x = x.fillna(x.mean())
test = test.fillna(test.mean())

In [56]:
x.shape

(10000, 35)

In [57]:
test.shape

(10000, 35)

In [58]:
train = np.array(train)
x_predict = np.array(test)

In [59]:
# 전처리
from sklearn.preprocessing import RobustScaler, StandardScaler
scaler = RobustScaler()
scaler.fit(x) 
x = scaler.transform(x)
x_predict = scaler.transform(x_predict)

# from sklearn.decomposition import PCA
# pca = PCA(n_components=12)
# pca.fit(x)
# x = pca.transform(x)
# x_predict = pca.transform(x_predict)

In [167]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(x) 
# x = scaler.transform(x)
# x_predict = scaler.transform(x_predict)

In [62]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.623793458333332


In [68]:
train = pd.read_csv('train.csv', index_col=0)
train = train.iloc[:,36:-4]

In [69]:
feature_importances_ = model.feature_importances_
feature_importances = pd.DataFrame({'랜덤포레스트_feature_importance':feature_importances_}, index=train.columns)
feature_importances = feature_importances.sort_values(by=['랜덤포레스트_feature_importance'],ascending=False)
feature_importances

,랜덤포레스트_feature_importance
780_dst,0.039314
790_dst,0.038471
850_dst,0.035945
970_dst,0.035426
810_dst,0.034453
770_dst,0.034297
960_dst,0.032369
980_dst,0.031985
800_dst,0.031917
830_dst,0.031843


In [ ]:
corr = x.corr()
Survived = corr.loc[:,['hhb','hbo2','ca','na']]
Survived = abs(Survived)
a = Survived.sort_values(by=['hhb','hbo2','ca','na'],
                              ascending=False)
a = a[2:]
a

In [2]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=160)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.4601283828124996


# GradientBoostingRegressor

In [9]:
y[:,0]

array([ 5.59,  0.  , 10.64, ...,  9.84,  6.38,  9.35])

In [10]:
a = []
for i in range(y.shape[1]) :
    a.append(y[:,i])

In [11]:
a

[array([ 5.59,  0.  , 10.64, ...,  9.84,  6.38,  9.35]),
 array([4.32, 2.83, 3.  , ..., 3.2 , 4.06, 4.34]),
 array([ 8.92,  7.25,  8.4 , ..., 10.45, 11.28,  9.73]),
 array([4.29, 4.64, 5.16, ..., 2.06, 4.03, 3.54])]

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x,a[0], test_size=0.2, random_state=43)

model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[1], test_size=0.2, random_state=43)
model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[2], test_size=0.2, random_state=43)
model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[3], test_size=0.2, random_state=43)
model = GradientBoostingRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

GradientBoostingRegressor mae 는 1.5838840045681222
GradientBoostingRegressor mae 는 0.7350616301783675
GradientBoostingRegressor mae 는 2.308885373741608
GradientBoostingRegressor mae 는 1.4267326043872253


In [13]:
x_train, x_test, y_train, y_test = train_test_split(x,a[0], test_size=0.2, random_state=43)

model = GradientBoostingRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[1], test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[2], test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[3], test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

In [20]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca.fit(y)
y1 = pca.transform(y)
x_train, x_test, y_train, y_test = train_test_split(x,y1, test_size=0.2, random_state=43)
model = GradientBoostingRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('GradientBoostingRegressor mae 는', mae)

C:\Users\bitcamp\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


랜덤포레스트 mae 는 1.837030398551365


# XGBRegressor()

In [73]:
y.iloc[:,0]

id
0        5.59
1        0.00
2       10.64
3        5.67
4       11.97
        ...  
9995    12.68
9996     8.46
9997     9.84
9998     6.38
9999     9.35
Name: hhb, Length: 10000, dtype: float64

In [74]:
x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,0], test_size=0.2, random_state=43)

model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,1], test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,2], test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,3], test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

xgboost mae 는 1.985841414503455
xgboost mae 는 0.8020689496445654
xgboost mae 는 2.473951157708168
xgboost mae 는 1.4655645578134058


In [97]:
x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,0], test_size=0.2, random_state=43)

model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,1], test_size=0.2, random_state=43)

model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,2], test_size=0.2, random_state=43)

model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,3], test_size=0.2, random_state=43)

model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

RandomForestRegressor mae 는 1.8973619999999998
RandomForestRegressor mae 는 0.7542828833333333
RandomForestRegressor mae 는 2.333656416666667
RandomForestRegressor mae 는 1.4125609166666668


In [85]:
# 첫번째
x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,0], test_size=0.2, random_state=43)

model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

thresholds = np.sort(model.feature_importances_)

print(thresholds)

for thresh in thresholds : # 컬럼수만큼 돈다! 빙글빙글
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    selection_x_train = selection.transform(x_train)
    # print(selection_x_train.shape) # 칼럼이 하나씩 줄고 있는걸 알 수 있음 (가장 중요 x를 하나씩 지우고 있음)
    
    selection_model = XGBRegressor()
    selection_model.fit(selection_x_train, y_train)

    select_x_test = selection.transform(x_test)
    x_pred = selection_model.predict(select_x_test)

    score = mean_absolute_error(y_test, x_pred)
    print('mae는',score)

RandomForestRegressor mae 는 1.8965933666666668
[0.01760668 0.01764651 0.01849684 0.01857285 0.01914372 0.0196167
 0.02013176 0.02031608 0.02057872 0.02086349 0.02087433 0.02101057
 0.0217065  0.02174618 0.02176691 0.02290204 0.0235749  0.02363538
 0.02365461 0.02492243 0.02604238 0.02610384 0.02639844 0.03023521
 0.03049992 0.0311826  0.0324361  0.0355022  0.03619014 0.03901171
 0.04314435 0.04646882 0.04795265 0.06272853 0.06733592]
mae는 1.985841414503455
mae는 1.9082601587754489
mae는 1.9410161343801022
mae는 1.8781339675986768
mae는 1.9140227403986456
mae는 1.9058803449630737
mae는 1.893496838222146
mae는 1.8699945471704007
mae는 1.9030650523728132
mae는 1.9036353793019056
mae는 1.9191212417805195
mae는 1.9458620796561241
mae는 1.9013976004278659
mae는 1.921917523881793
mae는 1.918742645225525
mae는 1.9204881935125588
mae는 1.9281053926229474
mae는 1.9069887832456824
mae는 1.9504726079297066
mae는 1.939090841883421
mae는 1.9040193865537645
mae는 1.9391908463770153
mae는 1.945283686898947
mae는 1.975695208

In [84]:
# 첫번째
x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,0], test_size=0.2, random_state=43)

model = XGBRegressor(n_estimators=300, learning_rate=0.01)
model.fit(x_train, y_train, verbose=True, eval_metric=['logloss','rmse'], eval_set=[(x_train,y_train),(x_test,y_test)]
        , early_stopping_rounds=20)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

thresholds = np.sort(model.feature_importances_)

print(thresholds)

for thresh in thresholds : # 컬럼수만큼 돈다! 빙글빙글
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    selection_x_train = selection.transform(x_train)
    # print(selection_x_train.shape) # 칼럼이 하나씩 줄고 있는걸 알 수 있음 (가장 중요 x를 하나씩 지우고 있음)
    
    selection_model = XGBRegressor()
    selection_model.fit(selection_x_train, y_train)

    select_x_test = selection.transform(x_test)
    x_pred = selection_model.predict(select_x_test)

    score = mean_absolute_error(y_test, x_pred)
    print('mae는',score)
#     print("Thresh=%.3f, n=%d, R2: %.2f%%" %(thresh, selection_x_train.shape[1], score*100.0))

[0]	validation_0-logloss:-1.62349	validation_0-rmse:7.98289	validation_1-logloss:-1.63345	validation_1-rmse:8.00147
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[1]	validation_0-logloss:-4.01836	validation_0-rmse:7.91206	validation_1-logloss:-4.04162	validation_1-rmse:7.93042
[2]	validation_0-logloss:-6.65726	validation_0-rmse:7.84202	validation_1-logloss:-6.69348	validation_1-rmse:7.86024
[3]	validation_0-logloss:-9.86237	validation_0-rmse:7.77273	validation_1-logloss:-9.91968	validation_1-rmse:7.79076
[4]	validation_0-logloss:-23.35875	validation_0-rmse:7.70420	validation_1-logloss:-23.24529	validation_1-rmse:7.72210
[5]	validation_0-logloss:-54.33299	validation_0-rmse:7.63644	validation_1-logloss:-55.22797	validation_1-rmse:7.65435
[6]	validation_0-logloss:-150.32570	validation_0-rmse:7.56950	validation_1-logloss:-155.65308	validation_1-rmse:7.58738
[7]	validation_0-log

[67]	validation_0-logloss:-257.47443	validation_0-rmse:4.64537	validation_1-logloss:-258.02139	validation_1-rmse:4.67468
[68]	validation_0-logloss:-257.47421	validation_0-rmse:4.61281	validation_1-logloss:-258.02133	validation_1-rmse:4.64244
[69]	validation_0-logloss:-257.47421	validation_0-rmse:4.58068	validation_1-logloss:-258.00555	validation_1-rmse:4.61123
[70]	validation_0-logloss:-257.47089	validation_0-rmse:4.54906	validation_1-logloss:-258.00555	validation_1-rmse:4.57996
[71]	validation_0-logloss:-257.47128	validation_0-rmse:4.51767	validation_1-logloss:-258.00555	validation_1-rmse:4.54915
[72]	validation_0-logloss:-257.47128	validation_0-rmse:4.48695	validation_1-logloss:-258.00555	validation_1-rmse:4.51883
[73]	validation_0-logloss:-257.47128	validation_0-rmse:4.45649	validation_1-logloss:-258.00555	validation_1-rmse:4.48881
[74]	validation_0-logloss:-257.47122	validation_0-rmse:4.42653	validation_1-logloss:-258.00555	validation_1-rmse:4.45951
[75]	validation_0-logloss:-257.4

[135]	validation_0-logloss:-257.45017	validation_0-rmse:3.17423	validation_1-logloss:-258.00232	validation_1-rmse:3.24923
[136]	validation_0-logloss:-257.45017	validation_0-rmse:3.16130	validation_1-logloss:-258.00232	validation_1-rmse:3.23701
[137]	validation_0-logloss:-257.45020	validation_0-rmse:3.14862	validation_1-logloss:-258.00232	validation_1-rmse:3.22494
[138]	validation_0-logloss:-257.45020	validation_0-rmse:3.13572	validation_1-logloss:-258.00232	validation_1-rmse:3.21316
[139]	validation_0-logloss:-257.45020	validation_0-rmse:3.12309	validation_1-logloss:-258.00232	validation_1-rmse:3.20162
[140]	validation_0-logloss:-257.45017	validation_0-rmse:3.11058	validation_1-logloss:-258.00232	validation_1-rmse:3.19006
[141]	validation_0-logloss:-257.45023	validation_0-rmse:3.09866	validation_1-logloss:-258.00232	validation_1-rmse:3.17891
[142]	validation_0-logloss:-257.45023	validation_0-rmse:3.08684	validation_1-logloss:-258.00232	validation_1-rmse:3.16786
[143]	validation_0-loglo

[203]	validation_0-logloss:-257.44632	validation_0-rmse:2.60006	validation_1-logloss:-258.00232	validation_1-rmse:2.73490
[204]	validation_0-logloss:-257.44678	validation_0-rmse:2.59506	validation_1-logloss:-258.00232	validation_1-rmse:2.73096
[205]	validation_0-logloss:-257.44678	validation_0-rmse:2.59021	validation_1-logloss:-258.00232	validation_1-rmse:2.72715
[206]	validation_0-logloss:-257.44623	validation_0-rmse:2.58537	validation_1-logloss:-258.00232	validation_1-rmse:2.72308
[207]	validation_0-logloss:-257.44623	validation_0-rmse:2.58028	validation_1-logloss:-258.00232	validation_1-rmse:2.71891
[208]	validation_0-logloss:-257.44626	validation_0-rmse:2.57563	validation_1-logloss:-258.00232	validation_1-rmse:2.71519
[209]	validation_0-logloss:-257.44498	validation_0-rmse:2.57062	validation_1-logloss:-258.00232	validation_1-rmse:2.71109
[210]	validation_0-logloss:-257.44498	validation_0-rmse:2.56586	validation_1-logloss:-258.00232	validation_1-rmse:2.70754
[211]	validation_0-loglo

[271]	validation_0-logloss:-257.44488	validation_0-rmse:2.35582	validation_1-logloss:-258.00232	validation_1-rmse:2.56274
[272]	validation_0-logloss:-257.44488	validation_0-rmse:2.35333	validation_1-logloss:-258.00232	validation_1-rmse:2.56126
[273]	validation_0-logloss:-257.44488	validation_0-rmse:2.34987	validation_1-logloss:-258.00232	validation_1-rmse:2.55977
[274]	validation_0-logloss:-257.44488	validation_0-rmse:2.34772	validation_1-logloss:-258.00232	validation_1-rmse:2.55839
[275]	validation_0-logloss:-257.44488	validation_0-rmse:2.34440	validation_1-logloss:-258.00232	validation_1-rmse:2.55688
[276]	validation_0-logloss:-257.44488	validation_0-rmse:2.34192	validation_1-logloss:-258.00232	validation_1-rmse:2.55566
[277]	validation_0-logloss:-257.44488	validation_0-rmse:2.33978	validation_1-logloss:-258.00232	validation_1-rmse:2.55450
[278]	validation_0-logloss:-257.44486	validation_0-rmse:2.33733	validation_1-logloss:-258.00232	validation_1-rmse:2.55322
[279]	validation_0-loglo

In [87]:
# 두번째
x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,1], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

thresholds = np.sort(model.feature_importances_)

print(thresholds)
import joblib
for thresh in thresholds : # 컬럼수만큼 돈다! 빙글빙글
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    selection_x_train = selection.transform(x_train)
    # print(selection_x_train.shape) # 칼럼이 하나씩 줄고 있는걸 알 수 있음 (가장 중요 x를 하나씩 지우고 있음)
    
    selection_model = XGBRegressor()
    selection_model.fit(selection_x_train, y_train)

    select_x_test = selection.transform(x_test)
    x_pred = selection_model.predict(select_x_test)

    score = mean_absolute_error(y_test, x_pred)
    print('mae는',score)
    
    modelpath = './model_save/mae' + str(score*100.0) + '.dat'
    joblib.dump(selection_model,modelpath)

xgboost mae 는 0.7531651
[0.02442299 0.02481295 0.02525549 0.0252763  0.02576097 0.02577619
 0.0257982  0.02594912 0.02632982 0.02634321 0.0268536  0.02708139
 0.027993   0.02801181 0.02842158 0.02847693 0.02861737 0.02882096
 0.02889177 0.02892448 0.02894274 0.02903805 0.02908792 0.02947847
 0.02950036 0.02951103 0.02967051 0.02974296 0.02984125 0.03004418
 0.03111436 0.03170254 0.03279263 0.03372616 0.03798869]
mae는 0.8020689496445654
mae는 0.798264695482254
mae는 0.8034914375400543
mae는 0.7946517977476119
mae는 0.798954010643959
mae는 0.8108075167059898
mae는 0.7944103201723098
mae는 0.8058862382936477
mae는 0.787364046974182
mae는 0.7981777908945084
mae는 0.7909177214813232
mae는 0.796801345667839
mae는 0.7996973411750794
mae는 0.7938885533928871
mae는 0.79654620749712
mae는 0.7963952490901948
mae는 0.7945233472824097
mae는 0.7817250055599212
mae는 0.8001232743072509
mae는 0.7984421026420594
mae는 0.7871124481964111
mae는 0.7863553144264221
mae는 0.7963276730537414
mae는 0.7912447420835494
mae는 0.7948658

In [88]:
model2 = joblib.load('./model_save/mae78.17250055599212.dat')

In [90]:
model2

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [93]:
x_train.shape

(8000, 35)

In [92]:
x_predict.shape

(10000, 35)

In [89]:
pred = model2.predict(x_predict)


ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34']
training data did not have the following fields: f31, f18, f25, f20, f24, f30, f29, f32, f19, f26, f21, f27, f34, f23, f28, f33, f22

In [ ]:
# 세번째
x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,2], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

In [78]:
x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,3], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

xgboost mae 는 1.891076366666667
xgboost mae 는 0.7551793
xgboost mae 는 2.3387733833333333
xgboost mae 는 1.4129589


In [77]:
x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,0], test_size=0.2, random_state=43)

model = XGBRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,1], test_size=0.2, random_state=43)
model = XGBRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,2], test_size=0.2, random_state=43)
model = XGBRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,y.iloc[:,3], test_size=0.2, random_state=43)
model = XGBRegressor(n_estimators=300)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

xgboost mae 는 1.9900127496957778
xgboost mae 는 0.811327539126873
xgboost mae 는 2.4954054609155656
xgboost mae 는 1.4874754696315526


In [30]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca.fit(y)
y1 = pca.transform(y)
x_train, x_test, y_train, y_test = train_test_split(x,y1, test_size=0.2, random_state=43)
model = XGBRegressor()
model.fit(x_train,y_train)
y_pred = model.predict(x_test) # 회귀던 분류던 사용할 수 있음
mae = mean_absolute_error(y_pred, y_test)
print('xgboost mae 는', mae)

xgboost mae 는 1.8154817496832283


# RandomForestRegressor

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x,a[0], test_size=0.2, random_state=43)

model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[1], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[2], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

x_train, x_test, y_train, y_test = train_test_split(x,a[3], test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_pred, y_test)
print('RandomForestRegressor mae 는', mae)

xgboost mae 는 1.289325675
xgboost mae 는 0.7111221249999999
xgboost mae 는 2.187418975
xgboost mae 는 1.407558775


In [33]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=160)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.4622248984374993


In [37]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=43)
model = RandomForestRegressor(n_estimators=200)
model.fit(x_train,y_train)
y_pred = model.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.7716422312499989


In [39]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=20)
model1 = RandomForestRegressor(n_estimators=250)
model1.fit(x_train,y_train)
y_pred = model1.predict(x_test) 
mae = mean_absolute_error(y_pred, y_test)
print('랜덤포레스트 mae 는', mae)

랜덤포레스트 mae 는 1.441592375


In [ ]:
pred = model1.predict(x_predict)
submission = pd.read_csv('sample_submission.csv', index_col=0)
submission['hhb'] = pred[:,0]
submission['hbo2'] = pred[:,1]
submission['ca'] = pred[:,2]
submission['na'] = pred[:,3]

('sample_submission.csv', index_col=0)

# 제출

In [42]:
# 제출
pred = model1.predict(x_predict)

pred = model1.predict(x_predict)
submission = pd.read_csv('sample_submission.csv', index_col=0)
submission['hhb'] = pred[:,0]
submission['hbo2'] = pred[:,1]
submission['ca'] = pred[:,2]
submission['na'] = pred[:,3]

submission.to_csv('submission0611_1.csv')

In [255]:
submission

,hhb,hbo2,ca,na
id,,,,
10000,7.330,3.861,7.608,2.890
10001,6.530,3.993,6.310,2.105
10002,11.439,4.057,9.335,3.524
10003,8.239,4.599,9.715,3.084
10004,8.279,3.615,9.837,4.130
...,...,...,...,...
19995,6.134,4.480,9.480,3.342
19996,7.258,4.027,9.550,3.275
19997,9.724,3.333,9.786,3.206
